### II ) Ajout des notes des compagnies  
df_compagnies.csv -> df_compagnies.csv

In [1]:
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
df_compagnies = pd.read_csv('df_compagnies.csv', sep='§', engine='python', index_col=0, encoding='utf-8')
df_compagnies

,Compagnie,link
0,Emirates,https://www.tripadvisor.fr/Airline_Review-d872...
1,AEGEAN,https://www.tripadvisor.fr/Airline_Review-d872...
2,Qatar Airways,https://www.tripadvisor.fr/Airline_Review-d872...
3,Turkish Airlines,https://www.tripadvisor.fr/Airline_Review-d872...
4,Singapore Airlines,https://www.tripadvisor.fr/Airline_Review-d872...
...,...,...
235,Nordwind Airlines,https://www.tripadvisor.fr/Airline_Review-d108...
236,Buddha Air,https://www.tripadvisor.fr/Airline_Review-d872...
237,Lucky Air,https://www.tripadvisor.fr/Airline_Review-d872...
238,Ernest,https://www.tripadvisor.fr/Airline_Review-d150...


In [3]:
DF = pd.DataFrame(columns=["Compagnie","link","Note globale","Espace pour les jambes","Confort des sièges",
                           "Divertissement à bord (Wi-Fi, film, télévision)","Expérience à bord",
                           "Service client","Rapport qualité/prix","Propreté","Enregistrement et embarquement",
                           "Restauration et boissons","Prix","Avis"])
DF

,Compagnie,link,Note globale,Espace pour les jambes,Confort des sièges,"Divertissement à bord (Wi-Fi, film, télévision)",Expérience à bord,Service client,Rapport qualité/prix,Propreté,Enregistrement et embarquement,Restauration et boissons,Prix,Avis


In [4]:
options = Options()
#options.headless = True

driver = webdriver.Firefox(options=options)

In [5]:
for i in range(len(df_compagnies)):
    
    comp_name, comp_url = df_compagnies.loc[i,:]
    dict_infos = {"Compagnie" : comp_name,
                  "link" : comp_url}

    driver.get(comp_url)
    wait_time_long = 5 + np.random.random(1)[0]*5
    print(i, comp_name, "\tloading page... wait ", round(wait_time_long,2))
    time.sleep(wait_time_long)

    # récupération des notes de la compagnie
    note_globale = driver.find_elements_by_xpath("//span[contains(@class,'flights-airline-review-page-overview-module-OverviewModule__review_num')]")
    if note_globale : note_globale = note_globale[0].text
    else : note_globale = None
    dict_infos['Note globale'] = note_globale

    block_notes = driver.find_elements_by_xpath("//div[contains(@class,'flights-airline-review-page-overview-module-OverviewModule__amenity')]/div")
    for b in block_notes:
        note_spec = b.find_elements_by_xpath(".//span[contains(@class,'ui_bubble_rating bubble')]")
        if note_spec : note_spec = note_spec[0].get_attribute("class").split('_')[-1]
        else : note_spec = None
        note_name = b.text
        dict_infos[note_name] = note_spec

    # prix gagnés
    infos = driver.find_elements_by_xpath("//div[contains(@class,'flights-airline-review-page-overview-module-OverviewModule__tca_wins')]")
    if infos : infos = infos[0].text.split('\n')
    else : infos = None
    dict_infos['Prix'] = infos

    # nombre d'avis
    num_reviews = driver.find_elements_by_xpath("//span[contains(@class,'flights-airline-review-page-overview-module-OverviewModule__num_reviews')]")
    if num_reviews : num_reviews = num_reviews[0].text
    else : num_reviews = None
    dict_infos["Avis"] = num_reviews

    DF = DF.append(pd.Series(dict_infos), ignore_index=True, sort=False)
    DF.to_csv('df_compagnies_enrichi.csv', sep='§', encoding='utf-8')

0 Emirates loading page... wait  6.3
1 AEGEAN loading page... wait  7.26
2 Qatar Airways loading page... wait  6.82
3 Turkish Airlines loading page... wait  7.98
4 Singapore Airlines loading page... wait  7.19
5 Lufthansa loading page... wait  6.79
6 Thai Airways loading page... wait  8.29
7 Norwegian loading page... wait  9.58
8 TAP Air Portugal loading page... wait  9.09
9 Aeroflot loading page... wait  6.79
10 Aer Lingus loading page... wait  9.08
11 Delta Air Lines loading page... wait  9.09
12 Cathay Pacific loading page... wait  8.56
13 EVA Air loading page... wait  5.42
14 Oman Air loading page... wait  5.45
15 American Airlines loading page... wait  9.09
16 Austrian Airlines loading page... wait  6.62
17 Vietnam Airlines loading page... wait  7.21
18 Vueling Airlines loading page... wait  9.81
19 Volotea loading page... wait  6.26
20 Etihad Airways loading page... wait  7.07
21 ANA (All Nippon Airways) loading page... wait  7.99
22 Royal Air Maroc loading page... wait  7.14
23 

183 Alba Star loading page... wait  7.16
184 Air KBZ loading page... wait  6.8
185 Yeti Airlines loading page... wait  6.82
186 Jetstar Pacific Airlines - Vietnam loading page... wait  8.73
187 Arkia Israel Airlines loading page... wait  9.08
188 Frontier Airlines loading page... wait  8.2
189 Georgian Airways loading page... wait  5.61
190 Boliviana de Aviación loading page... wait  5.59
191 Sun Air Express loading page... wait  5.61
192 LATAM Brasil loading page... wait  7.17
193 Surinam Airways loading page... wait  5.65
194 TUI Airlines Netherlands loading page... wait  6.58
195 Caribbean Airlines loading page... wait  9.14
196 S7 Airlines loading page... wait  7.83
197 LCPeru loading page... wait  5.81
198 Mahan Air loading page... wait  6.11
199 Pobeda loading page... wait  5.13
200 Mann Yadanarpon Airlines loading page... wait  6.54
201 Shandong Airlines loading page... wait  7.95
202 Cathay Dragon (Dragonair) loading page... wait  5.55
203 Sriwijaya Air loading page... wait  8.

In [6]:
DF

,Compagnie,link,Note globale,Espace pour les jambes,Confort des sièges,"Divertissement à bord (Wi-Fi, film, télévision)",Expérience à bord,Service client,Rapport qualité/prix,Propreté,Enregistrement et embarquement,Restauration et boissons,Prix,Avis
0,Emirates,https://www.tripadvisor.fr/Airline_Review-d872...,"4,0",40,40,45,40,40,40,45,45,40,"[Lauréat Travellers' Choice 2019, Lauréat Trav...",58 244 avis
1,AEGEAN,https://www.tripadvisor.fr/Airline_Review-d872...,"4,0",40,40,30,40,45,40,45,45,40,"[Lauréat Travellers' Choice 2019, Lauréat Trav...",10 946 avis
2,Qatar Airways,https://www.tripadvisor.fr/Airline_Review-d872...,"4,0",40,40,40,40,45,40,45,45,40,"[Lauréat Travellers' Choice 2019, Lauréat Trav...",28 359 avis
3,Turkish Airlines,https://www.tripadvisor.fr/Airline_Review-d872...,"4,0",40,40,40,40,40,40,40,40,40,"[Lauréat Travellers' Choice 2019, Lauréat Trav...",28 424 avis
4,Singapore Airlines,https://www.tripadvisor.fr/Airline_Review-d872...,"4,5",40,40,45,40,45,40,45,45,40,"[Lauréat Travellers' Choice 2019, Lauréat Trav...",23 789 avis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Nordwind Airlines,https://www.tripadvisor.fr/Airline_Review-d108...,"3,0",35,30,15,20,35,30,35,35,25,None,1 196 avis
236,Buddha Air,https://www.tripadvisor.fr/Airline_Review-d872...,"3,5",35,35,20,30,35,35,35,30,25,None,181 avis
237,Lucky Air,https://www.tripadvisor.fr/Airline_Review-d872...,"3,0",30,30,20,25,30,35,35,35,20,None,47 avis
238,Ernest,https://www.tripadvisor.fr/Airline_Review-d150...,"2,0",25,25,20,25,25,25,30,30,25,None,396 avis
